In [ ]:
#@title ## Configurations

#@markdown Enter a string that'll be used as the root user password
rootPassword = "password" #@param {type:"string"}


#@markdown Select the tools to be installed during bootstrap
Docker = False #@param {type:"boolean"}
Neo4j = False #@param {type:"boolean"}
Jekyll = True #@param {type:"boolean"}

#@markdown Select whether if you want access to Code Server or SSH Server or both
typeOfAccess = "Code Server" #@param ["Code Server", "SSH Server", "Both"]

#@markdown Enter the URL of the docker playground instance that'll be used as the remote docker machine. **Required only while using docker**
dockerPlaygroundUrl = "" #@param {type:"string"}

import os
os.environ['rootPassword'] = rootPassword
os.environ['Docker'] = str(Docker)
os.environ['Neo4j'] = str(Neo4j)
os.environ['Jekyll'] = str(Jekyll)
os.environ['typeOfAccess'] = typeOfAccess
os.environ['dockerPlaygroundUrl'] = dockerPlaygroundUrl

# System Specifications

Describe the Google Compute Engine backend powering this google-colab session.

In [ ]:
# Linux kernel version
!uname -a

In [ ]:
# OS name and version
!cat /etc/lsb-release

In [ ]:
# CPU architecture information
!lscpu

In [ ]:
# Check disk space used by filesystem 
!df -h

In [ ]:
# Check amount of free and used memory (both physical and swap)
# on the system as well as the shared memory and buffers used by the kernel
!cat /proc/meminfo

# Who am I?

Get the user associated with the current effective user ID and update/set-new password.

In [ ]:
# Who am I?
!whoami

In [ ]:
# Set root password auth
! echo root:$rootPassword | chpasswd
# ! mkdir -p /var/run/sshd
# ! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
# ! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
# ! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
# ! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

# Environment Setup

Create your own collection of procedures and tools for developing, testing and debugging a program or complete application.

In [ ]:
%%shell

# Clone dotfiles repository - https://github.com/adisakshya/dotfiles
git clone --recursive https://github.com/adisakshya/dotfiles ~/.dotfiles
chmod +x ~/.dotfiles/install-profile

# Install dotfiles for linux
make linux -C ~/.dotfiles

# Bootstrap remote environment
make bootstrap -C ~/.dotfiles/remote

In [ ]:
# Create directory to contain log files
!mkdir logs

In [ ]:
%%shell

if [[ "$Docker" == "True" ]]
then
    echo "Please install manually using '$ ins docker' form code-server shell"
fi

if [[ "$Neo4j" == "True" ]]
then
    echo "Installing Neo4j"
    make neo4j -C ~/.dotfiles/remote
fi

if [[ "$Jekyll" == "True" ]]
then
    echo "Installing Jekyll"
    make jekyll -C ~/.dotfiles/remote
fi

# Setup Code Server

Install and setup [code-server](https://github.com/cdr/code-server).

Starting code-server - A password will be generated for you by default which can be found at ```~/.config/code-server/config.yaml```.

Accessing code-server - By default code-server runs on localhost and needs to be securely exposed to the internet to be able to access it from a web browser. 

[Localtunnel](https://github.com/localtunnel/localtunnel) creates a tunnel to the specified local port offering secure https for all tunnels. **The tunnel-url for code-server is stored in ```logs/cdr_tunnel.out```**. This url will remain active for the duration of your session; so feel free to share it with others for peer programming.

In [ ]:
%%shell

if [[ "$typeOfAccess" == "Code Server" || "$typeOfAccess" == "Both" ]]
then
    # Install code server
    make code -C ~/.dotfiles/remote

    # redirect standard output and standard error to different files
    # logs can be found in the logs/ directory
    nohup code-server --port 8000 > ./logs/code_server.out 2> ./logs/code_server.err &

    # Start localtunnel - redirect standard output and standard error to different files
    # logs can be found in the logs/ directory
    nohup lt --port 8000 > logs/cdr_tunnel.out 2> logs/cdr_tunnel.err &
fi

In [ ]:
%%shell

if [[ "$typeOfAccess" == "Code Server" || "$typeOfAccess" == "Both" ]]
then
    echo ">> Show code-server config"
    cat ~/.config/code-server/config.yaml
    echo ">> Show code-server logs"
    cat logs/code_server.out
    echo ">> Show localtunnel logs => tunnel-url"
    cat logs/cdr_tunnel.out
fi

# OpenSSH server process

If you want to have a terminal based system with SSH access then consider running this section.

Prerequisites:
1. Updated/set-new user password (Section 2 - Who am I?)
2. Ngrok is installed (Section 3 - Environment Setup)
    - If you havn't executed section 3 then consider installing ngrok before proceeding further

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

if typeOfAccess not in ["SSH Server", "Both"]:
    print('SSH Server is not requred .. thus stopping execution')
    raise StopExecution

In [ ]:
# Setup sshd
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

In [ ]:
# Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

1. ngrok can create a tunnel to this system and give us a publicly accessible hostname. 
2. You will need to login/sign-up, copy your auth token from ngrok dashboard and provide it in the next cell when prompted.

In [ ]:
# Set auth-token
import getpass
print("Copy your authtoken from https://dashboard.ngrok.com/auth")
authtoken = getpass.getpass()

In [ ]:
# Create ngrok tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

In [ ]:
# Get public address and print ssh connect command
import urllib.request, json
with urllib.request.urlopen('http://localhost:4040/api/tunnels') as response:
    data = json.loads(response.read().decode())
    (host, port) = data['tunnels'][0]['public_url'][6:].split(':')
    print(f'SSH command: ssh -p {port} root@{host}')
print('Use root password that you update/created in cell 7')

If the above cell fails, try running the cell again or run the next cell which will provide you the user, hostname and port.

In [ ]:
!echo 'User: root'
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
!echo 'Use root password that you update/created in cell 7'